### RankForecasting-shortterm-paper

test 1k pitmodel

basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator_shortterm_pitmodel as stint
import indycar.model.stint_simulator_paper as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0, exproot='remote'):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    #predictor = stint.load_model(predictionlen, model,trainid='2018',exproot=exproot)
    predictor = stint.load_model(predictionlen, model,trainid='indy500',exproot=exproot)

    ret2 = {}
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [3]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', 
                  inlapmode=0,featuremode = stint.FEATURE_PITAGE,
                 predictionlen=2, exproot='remote'):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    #dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
    dataset = 'indy2013-2018-%s-%s'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    #try:
    if True:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       predictionlen, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode,
                                 exproot=exproot)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           predictionlen, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode,
                                 exproot=exproot)
    #except:
    #    acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [4]:
def get_samples(startlap, carno=12):
    samples = ret['pitmodel-RANK-2018-inlap-nopitage'].keys()
    retdf = []
    for id in samples:
        df = ret['pitmodel-RANK-2018-inlap-nopitage'][id]
        rec = df[(df['carno']==carno) & (df['startlap']==startlap)]
        retdf.append(rec)
        
    dfout = pd.concat(retdf)
    return dfout



def get_alldf(year=2018, model='pitmodel'):
    

    dfx = ret[f'{model}-RANK-{year}-inlap-nopitage']
    #dfx = ret[f'{model}-TIMEDIFF-{year}-noinlap-nopitage']
    
    samples = dfx.keys()
    retdfs = []
    for id in samples:
        df = dfx[id][0]
        retdfs.append(df)
        
    if len(retdfs) > 1:
        dfout = pd.concat(retdfs)
    else:
        dfout = retdfs[0]
        
    return dfout
    
def get_alldf_mode(year=2018,mode=0,model='pitmodel'):
    """
    mode: 
        0; mode
        1; mean
        2; median
    """
    dfall = get_alldf(year=year, model=model)
    
    cars = set(dfall.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(dfall[dfall['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:
            dfrec = dfall[(dfall['carno']==car) & (dfall['startlap']==startlap)]
            
            #get mode
            if mode == 0:
                pred_endrank = stats.mode(dfrec.pred_endrank.values).mode[0]
                #pred_endlap =  stats.mode(dfrec.pred_endlap.values).mode[0]
            elif mode == 1:
                #use mean
                pred_endrank = np.mean(dfrec.pred_endrank.values)
                #pred_endlap =  np.mean(dfrec.pred_endlap.values)
            elif mode == 2:
                #use mean
                pred_endrank = np.median(dfrec.pred_endrank.values)
                #pred_endlap =  np.median(dfrec.pred_endlap.values)
            
            firstrec = dfrec.to_numpy()[0,:]
            firstrec[6] = pred_endrank
            firstrec[7] = pred_endrank - firstrec[2]
            if firstrec[7] == 0:
                firstrec[8] = 0
            elif firstrec[7] > 0:
                firstrec[8] = 1
            else:
                firstrec[8] = -1
                
            #endlap, pred_endlap
            
        
            retdf.append(firstrec)
        
    #dfout = pd.concat(retdf)
    dfout = pd.DataFrame(retdf, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         #'endlap','pred_endlap'
                                        ])
    print('df size:', len(dfout))
    return dfout

In [5]:
def get_prisk_ex(year=2018, model='pitmodel'):
    dfx = ret[f'{model}-RANK-{year}-inlap-nopitage']
    allsamples, alltss = stint.get_allsamples_ex(dfx)
    _, prisk_vals =stint.prisk_direct_bysamples(allsamples, alltss)
    
    return prisk_vals

def get_prisk(ret, mid):
    dfx = ret[mid]
    allsamples, alltss = stint.get_allsamples_ex(dfx)
    _, prisk_vals =stint.prisk_direct_bysamples(allsamples, alltss)
    
    return prisk_vals

def get_best(pitmodel=True, verbose=False):
    best_acc = {2018:[0, ''], 2019:[0,'']}
    best_mae = {2018:[100, ''], 2019:[100,'']}
    for model in ret.keys():
        if model.find('pitmodel')==0:
            if len(ret[model]) > 0:
                oracle = model.replace('pitmodel','oracle')
                if verbose:
                    print('model:',(oracle))
                acc_oracle = np.mean(acc[oracle], axis=0)
                if verbose:
                    print(acc_oracle)

                if verbose:
                    print('model:',(model))
                acc_model = np.mean(acc[model], axis=0)
                if verbose:
                    print(acc_model)

                if model.find('2018') > 0:
                    year = 2018
                else:
                    year = 2019
                    
                if pitmodel:
                    test_model = acc_model
                    modelname = model
                else:
                    test_model = acc_oracle
                    modelname = oracle
                    
                #mae
                if test_model[0, 0] > best_acc[year][0]:
                    best_acc[year][0] = test_model[0, 0]
                    best_acc[year][1] = modelname

                if test_model[0, 1] < best_mae[year][0]:
                    best_mae[year][0] = test_model[0, 1]
                    best_mae[year][1] = modelname
                
    #print('mae',best_mae)
    #print('acc',best_acc)                
    return best_acc, best_mae

In [6]:
import pickle
def save_result(dfs, datafile):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [dfs]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### simulation with 100 runs

In [7]:
#_exproot = 'remote'
_exproot = 'ranknet-train'
_predictionlen = 4

In [8]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
acc, ret, pret = {}, {}, {}

In [9]:
#
#  pitmodel rank-inlap-nopitage
# 

for year in ['2018','2019']:
    lapmode = 1
    fmode = stint.FEATURE_STATUS
    runts = 'RANK'
    mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
    acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=100,inlapmode=lapmode,featuremode = fmode,
                                      predictionlen=_predictionlen, exproot=_exproot)



/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model


model: acc={0.69}, mae={2.16}, rmse={4.12},r2={0.76}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.67}, mae={2.13}, rmse={4.07},r2={0.77}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.69}, mae={2.13}, rmse={4.10},r2={0.76}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.70}, mae={2.13}, rmse={4.07},r2={0.77}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.69}, mae={2.16}, rmse={4.13},r2={0.76}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.70}, mae={2.12}, rmse={4.06},r2={0.77}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.70}, mae={2.13}, rmse={4.09},r2={0.77}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.69}, mae={2.14}, rmse={4.09},r2={0.77}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=

model: acc={0.66}, mae={1.74}, rmse={3.73},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.68}, mae={1.74}, rmse={3.74},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.66}, mae={1.73}, rmse={3.71},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.68}, mae={1.73}, rmse={3.72},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.65}, mae={1.74}, rmse={3.72},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.65}, mae={1.75}, rmse={3.74},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.69}, mae={1.74}, rmse={3.72},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.67}, mae={1.74}, rmse={3.74},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}


In [10]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [11]:
#
# oracle rank-inlap-nopitage
#
for year in ['2018','2019']:
    lapmode = 1
    fmode = stint.FEATURE_STATUS
    runts = 'RANK'
    mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
    acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=10,inlapmode=lapmode,
                                       featuremode = fmode,predictionlen=_predictionlen, exproot=_exproot)
    

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
model: acc={0.71}, mae={2.03}, rmse={3.92},r2={0.78}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.75}, mae={2.03}, rmse={3.90},r2={0.79}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.74}, mae={2.03}, rmse={3.91},r2={0.79}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.72}, mae={2.04}, rmse={3.91},r2={0.79}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.75}, mae={2.02}, rmse={3.89},r2={0.79}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.75}, mae={2.02}, rmse={3.90},r2={0.79}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.75}, mae={2.02}, rmse={3.88},r2={0.79}, {186}
            naive: 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.73}, mae={1.67}, rmse={3.58},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.71}, mae={1.67}, rmse={3.59},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.69}, mae={1.66}, rmse={3.58},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.71}, mae={1.69}, rmse={3.60},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.70}, mae={1.66}, rmse={3.57},r2={0.83}, {186}

In [12]:
### get prisk
for mid in ret:
    prisk_vals = get_prisk(ret, mid)
    pret[mid] = prisk_vals
    print('mid:',mid, 'prisk:',prisk_vals)

sacmplecnt: 100 lapcnt: 200 runcnt: 100


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3534: RuntimeWarning: Mean of empty slice
  forecast_mean = np.nanmean(forecast[carno], axis=0)
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3281: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.12091629471522752, 0.14358532367880683, 0.12898180049713606])
mid: pitmodel-RANK-2018-inlap-nopitage prisk: [0.12091629 0.14358532 0.1289818 ]
sacmplecnt: 100 lapcnt: 200 runcnt: 100
dict_values([0.09481836148875616, 0.11191004922618397, 0.09555897003676697])
mid: pitmodel-RANK-2019-inlap-nopitage prisk: [0.09481836 0.11191005 0.09555897]
sacmplecnt: 100 lapcnt: 200 runcnt: 10
dict_values([0.13218696909110558, 0.13876040203177348, 0.12641194747649415])
mid: oracle-RANK-2018-inlap-nopitage prisk: [0.13218697 0.1387604  0.12641195]
sacmplecnt: 100 lapcnt: 200 runcnt: 10
dict_values([0.10271227967312836, 0.10893320874100673, 0.09753698315560605])
mid: oracle-RANK-2019-inlap-nopitage prisk: [0.10271228 0.10893321 0.09753698]


In [13]:
dfs={}
for model in ['pitmodel','oracle']:
    #for model in ['pitmodel']:
    for year in ['2018','2019']:
        for mode in [0,1,2]:
        
            df = get_alldf_mode(year=year,mode=mode,model=model)
            name = '%s_%s'%(model, 'mean' if mode==1 else ('mode' if mode==0 else 'median'))
            if year not in dfs:
                dfs[year] = {}
            dfs[year][name] = df

_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''            
outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-t{_predictionlen}-alldata.pickle'
save_result([dfs,acc,ret,pret], outfile)

df size: 5274
df size: 5274
df size: 5274
df size: 5565
df size: 5565
df size: 5565
df size: 5274
df size: 5274
df size: 5274
df size: 5565
df size: 5565
df size: 5565


### parameter tuning

In [19]:
acc, ret, pret = {}, {}, {}

In [20]:
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,
                                                   inlapmode=lapmode,featuremode = fmode,
                                                  predictionlen=_predictionlen, exproot=_exproot)



INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
model: acc={0.75}, mae={1.48}, rmse={2.97},r2={0.88}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
[[0.74731183 1.47516117 2.97040133 0.87630093]
 [0.59677419 2.27152067 4.43543835 0.72419047]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
model: acc={0.74}, mae={1.69}, rmse={3.31},r2={0.85}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
[[0.73655914 1.69093667 3.3141656  0.84601281]
 [0.59677419 2.27152067 4.43543835 0.72419047]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
model: acc={0.66}, mae={1.73}, rmse={3.69},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.66129032 1.73333333 3.69005852 0.82191357]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
model: acc={0.69}, mae={1.68}, rmse={3.61},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.68817204 1.68409704 3.60751935 0.82979134]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
model: acc={0.68}, mae={1.69}, rmse={3.59},r2={0.83}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.67741935 1.69092543 3.59074382 0.83137065]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


In [22]:
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,
                                                   featuremode = fmode,predictionlen=_predictionlen, exproot=_exproot)


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
model: acc={0.65}, mae={2.40}, rmse={4.53},r2={0.71}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.66}, mae={2.27}, rmse={4.28},r2={0.74}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.65}, mae={2.45}, rmse={4.56},r2={0.71}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
[[0.65232975 2.37214006 4.45543418 0.72147728]
 [0.59677419 2.27152067 4.43543835 0.72419047]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
model: acc={0.61}, mae={2.48}, rmse={4.56},r2={0.71}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.64}, mae={2.52}, rmse={4.63},r2={0.70}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
model: acc={0.60}, mae={2.49}, rmse={4.60},r2={0.70}, {186}
            naive: acc={0.60}, mae={2.27}, rmse={4.44},r2={0.72}
[[0.61648746 2.49677664 4.59496542 0.70398421]
 [0.59677419 2.27152067 4.43543835 0.72419047]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.66}, mae={1.80}, rmse={3.81},r2={0.81}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.66}, mae={1.78}, rmse={3.79},r2={0.81}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.63}, mae={1.79}, rmse={3.81},r2={0.81}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.64874552 1.78987721 3.80178873 0.81096492]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.67}, mae={1.75}, rmse={3.74},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.68}, mae={1.74}, rmse={3.73},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.67}, mae={1.75}, rmse={3.74},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.67204301 1.74591195 3.73718432 0.81733544]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=4
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
model: acc={0.62}, mae={1.74}, rmse={3.72},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.63}, mae={1.72}, rmse={3.70},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
model: acc={0.65}, mae={1.71}, rmse={3.68},r2={0.82}, {186}
            naive: acc={0.58}, mae={1.90}, rmse={4.12},r2={0.78}
[[0.63082437 1.72410901 3.70225292 0.82073065]
 [0.57526882 1.89577718 4.11606106 0.77842134]]


In [23]:
### get prisk
for mid in ret:
    prisk_vals = get_prisk(ret, mid)
    pret[mid] = prisk_vals
    print('mid:',mid, 'prisk:',prisk_vals)

sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.11412109586080191, 0.11412109586080191, 0.11412109586080191])
mid: oracle-TIMEDIFF-2018-noinlap-pitage prisk: [0.1141211 0.1141211 0.1141211]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.12793229222954716, 0.12793229222954716, 0.12793229222954716])
mid: oracle-TIMEDIFF-2018-inlap-pitage prisk: [0.12793229 0.12793229 0.12793229]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.12275099967578083, 0.12275099967578078, 0.12275099967578083])
mid: oracle-TIMEDIFF-2018-outlap-pitage prisk: [0.122751 0.122751 0.122751]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.10366521308952327, 0.10366521308952326, 0.10366521308952326])
mid: oracle-TIMEDIFF-2019-noinlap-pitage prisk: [0.10366521 0.10366521 0.10366521]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.10619614740982326, 0.10619614740982326, 0.10619614740982324])
mid: oracle-TIMEDIFF-2019-inlap-pitage prisk: [0.10619615 0.10619615 0.10619615]
sacmplecnt: 100 lapcnt: 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3535: RuntimeWarning: Mean of empty slice
  full_samples[carno][runid, :] = forecast_mean
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_paper.py:3282: RuntimeWarning: invalid value encountered in less_equal
  )


dict_values([0.09678429648087752, 0.09678429648087751, 0.09678429648087752])
mid: oracle-TIMEDIFF-2019-noinlap-nopitage prisk: [0.0967843 0.0967843 0.0967843]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.09348699720278014, 0.09348699720278014, 0.09348699720278011])
mid: oracle-TIMEDIFF-2019-inlap-nopitage prisk: [0.093487 0.093487 0.093487]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.0972416236090244, 0.09724162360902441, 0.0972416236090244])
mid: oracle-TIMEDIFF-2019-outlap-nopitage prisk: [0.09724162 0.09724162 0.09724162]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.1141856695125905, 0.1141856695125905, 0.1141856695125905])
mid: oracle-RANK-2018-noinlap-pitage prisk: [0.11418567 0.11418567 0.11418567]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.11495298822003676, 0.11495298822003676, 0.11495298822003673])
mid: oracle-RANK-2018-inlap-pitage prisk: [0.11495299 0.11495299 0.11495299]
sacmplecnt: 100 lapcnt: 200 runcnt: 1
dict_values([0.1195485248027666

In [24]:
get_best(pitmodel=True, verbose=False)

({2018: [0.6881720430103827, 'pitmodel-RANK-2018-noinlap-pitage'],
  2019: [0.6971326164870804, 'pitmodel-RANK-2019-inlap-pitage']},
 {2018: [1.9102515484768043, 'pitmodel-RANK-2018-inlap-pitage'],
  2019: [1.6246780473195568, 'pitmodel-RANK-2019-outlap-pitage']})

In [25]:
get_best(pitmodel=False, verbose=False)

({2018: [0.7956989247307551, 'oracle-RANK-2018-outlap-pitage'],
  2019: [0.7903225806447365, 'oracle-RANK-2019-noinlap-pitage']},
 {2018: [1.3428138035646568, 'oracle-TIMEDIFF-2018-noinlap-nopitage'],
  2019: [1.2287511230907457, 'oracle-TIMEDIFF-2019-inlap-nopitage']})

In [26]:
outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-t{_predictionlen}-tuning.pickle'
save_result([[],acc,ret,pret], outfile)